In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import os
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNet

In [23]:
data_path = '/Users/asish/Documents/Projects/Internship project/fruits-360/Training'  
fruit_names = []
fruit_images = []
max_images_per_class = 50  # Start with only 50 pictures per fruit to save memory

# Load pictures
for folder in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder)
    if os.path.isdir(folder_path):
        count = 0
        for file in os.listdir(folder_path):
            if count >= max_images_per_class:
                break
            img_path = os.path.join(folder_path, file)
            try:
                img = cv2.imread(img_path)
                if img is None:
                    print(f"Could not read image: {img_path}")
                    continue
                img = cv2.resize(img, (224, 224))
                fruit_images.append(img)
                fruit_names.append(folder)
                count += 1
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
                continue

# Check if images were loaded
if len(fruit_images) == 0:
    print("No images loaded! Check your data_path or dataset.")
    exit()
else:
    print(f"Loaded {len(fruit_images)} images successfully!")

# Turn pictures and names into numbers
fruit_images = np.array(fruit_images) / 255.0
label_encoder = LabelEncoder()
fruit_labels = label_encoder.fit_transform(fruit_names)

Loaded 10300 images successfully!


In [24]:
X_train, X_test, y_train, y_test = train_test_split(fruit_images, fruit_labels, test_size=0.2, random_state=42)

In [25]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
258/258 ━━━━━━━━━━━━━━━━━━━━ 86s 330ms/step - accuracy: 0.2133 - loss: 4.0808 - val_accuracy: 0.7403 - val_loss: 1.1467
Epoch 2/5
258/258 ━━━━━━━━━━━━━━━━━━━━ 83s 320ms/step - accuracy: 0.8503 - loss: 0.7644 - val_accuracy: 0.8917 - val_loss: 0.4547
Epoch 3/5
258/258 ━━━━━━━━━━━━━━━━━━━━ 91s 354ms/step - accuracy: 0.9413 - loss: 0.3053 - val_accuracy: 0.9432 - val_loss: 0.2510
Epoch 4/5
258/258 ━━━━━━━━━━━━━━━━━━━━ 98s 379ms/step - accuracy: 0.9713 - loss: 0.1595 - val_accuracy: 0.9665 - val_loss: 0.1650
Epoch 5/5
258/258 ━━━━━━━━━━━━━━━━━━━━ 92s 357ms/step - accuracy: 0.9822 - loss: 0.1001 - val_accuracy: 0.9689 - val_loss: 0.1319


In [26]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"The computer got {test_accuracy * 100:.2f}% correct!")

65/65 ━━━━━━━━━━━━━━━━━━━━ 15s 236ms/step - accuracy: 0.9730 - loss: 0.1220
The computer got 96.89% correct!


In [11]:
import matplotlib as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [15]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
cm = confusion_matrix(y_test, y_pred_classes)
cm

65/65 ━━━━━━━━━━━━━━━━━━━━ 14s 220ms/step


array([[ 8,  0,  0, ...,  0,  0,  0],
       [ 0, 11,  0, ...,  0,  0,  0],
       [ 0,  0, 10, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  7,  0,  0],
       [ 0,  0,  0, ...,  0,  7,  0],
       [ 0,  0,  0, ...,  0,  0, 10]])

In [37]:
new_img_path = '/Users/asish/Documents/Projects/Internship project/newImage.jpg'  # CHANGE to your picture path
try:
    new_img = cv2.imread(new_img_path)
    if new_img is None:
        print(f"Could not read image: {new_img_path}")
    else:
        new_img = cv2.resize(new_img, (224, 224))
        new_img = np.array(new_img) / 255.0
        new_img = np.expand_dims(new_img, axis=0)
        prediction = model.predict(new_img)
        predicted_class = np.argmax(prediction, axis=1)[0]
        predicted_fruit = label_encoder.classes_[predicted_class]
        print(f"The computer thinks this is a {predicted_fruit}!")
        plt.imshow(cv2.cvtColor(new_img[0], cv2.COLOR_BGR2RGB))
        plt.title(f'Predicted: {predicted_fruit}')
        plt.savefig('predicted_apple.jpg')
        plt.show()
except Exception as e:
    print(f"Error loading {new_img_path}: {e}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
The computer thinks this is a Pear Red 1!
Error loading /Users/asish/Documents/Projects/Internship project/newImage.jpg: OpenCV(4.11.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.simd_helpers.hpp:94: error: (-2:Unspecified error) in function 'cv::impl::(anonymous namespace)::CvtHelper<cv::impl::(anonymous namespace)::Set<3, 4>, cv::impl::(anonymous namespace)::Set<3, 4>, cv::impl::(anonymous namespace)::Set<0, 2, 5>>::CvtHelper(InputArray, OutputArray, int) [VScn = cv::impl::(anonymous namespace)::Set<3, 4>, VDcn = cv::impl::(anonymous namespace)::Set<3, 4>, VDepth = cv::impl::(anonymous namespace)::Set<0, 2, 5>, sizePolicy = cv::impl::(anonymous namespace)::NONE]'
> Unsupported depth of input image:
>     'VDepth::contains(depth)'
> where
>     'depth' is 6 (CV_64F)

